# imports:

In [1]:
from os import listdir,mkdir,makedirs
from os.path import isfile, join, isdir,exists
import pandas as pd
import numpy as np
from scipy import stats
import re
import time
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from myplots import roundup, rounddown, find_decimal_fold, percentile_cut_off, rarefaction_calc, rarefaction_plot,draw_correlation_scatter
from matplotlib.ticker import FormatStrFormatter
import cPickle as pickle
from Bio.SeqUtils import GC
import seaborn as sns
import random
from scipy.stats import pearsonr,fisher_exact,ttest_ind,mannwhitneyu, sem
from skbio.diversity.alpha import shannon, simpson, berger_parker_d

from pop_organize import get_sample_data, get_sample_with_dfs
from SufficientStatistics import *
from MyFunctionsShani import *
import math
from myplots import roundup, rounddown, find_decimal_fold
from skbio.stats.distance import mantel
from scipy.spatial.distance import braycurtis, pdist, euclidean


from GeneralFeaturePhenotypeInteractions.Feature_phenotype_functions import * 
from TCR_microbiome_interactions.TCR_microbiome_interactions_functions import *
from TCR_microbiome_interactions.TCR_microbiome_interactions_functions2 import *
from SampleLists.SampleFileFunctions import *
from PhenotypicData.PhenotypeGenerationFunctions import *
from CardioProject.CardioFunctions import *
from PredictionPipeline.PredictionFunctions import *
from TCR_feature_generation.SubsamplingFunctions import *

import os
from Utils import cacheOnDisk, Load
from queue.qp import qp,fakeqp
from addloglevels import sethandlers

#ML imports:
from xgboost import XGBClassifier
import lightgbm as lgb
from collections import OrderedDict
from sklearn.model_selection import GroupKFold,StratifiedKFold, KFold
import statsmodels.formula.api as sm
from sklearn.linear_model import LogisticRegression, LinearRegression
import shap
from sklearn import metrics, preprocessing
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel,SelectKBest,chi2,mutual_info_classif,f_classif
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import *

MyPath='/net/mraid08/export/genie/Lab/Personal/ShaniBAF'

with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/PNP530','rb') as fp:
    PNP530=pickle.load(fp)
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/Cardio126','rb') as fp:
    Cardio126=pickle.load(fp)
PNP530Cardio126=PNP530+Cardio126

pd.set_option('display.expand_frame_repr', False)


done1
stop
stop
done1
stop
done
done
done
done
stop
stop
end of function!!!
end of function!!!


# definitions:

In [2]:
#####path definitions:
MyPath = '/net/mraid08/export/genie/Lab/Personal/ShaniBAF'

with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/PNP530', 'rb') as fp:
    PNP530 = pickle.load(fp)
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/Cardio126', 'rb') as fp:
    Cardio126 = pickle.load(fp)
PNP530Cardio126 = PNP530 + Cardio126

PRED_RESULTS_DIR='/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/predictions2/'
FEATURES_DIR='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Predictions/featureDFs/'
SAMPLE_LIST_DIR='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/'
CARDIO_PHEN_DIR='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/CardioSamples/phenotypicData/'
CLUSTER_ANALYSIS_DIR='/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/sharingAnalysis/seqClusters_allProd_maxdist1/'

NEW_FEATURES_DIR='/net/mraid08/export/jafar/Microbiome/Analyses/TCR/'
SEASON_DIR='%s/TCR_real_data/seasonality/' %MyPath

##### general definitions:
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.width', 1000)
np.set_printoptions(precision=4, linewidth=200)

cdate = str(time.strftime("%d%m%Y"))
cdate

'22042019'

# extract dates for patients and controls:

## pnp530:

### generate table with months and weeks

In [ ]:
BDtable=pd.read_excel('%s/Sample files/updatedBDandFDlists/BDfile_31052018.xlsx' %MyPath)
dates=BDtable[['DnaID','Blood_Date']].set_index('DnaID').loc[PNP530,:].dropna()
print dates.shape
dates['Blood_Date']=pd.to_datetime(dates['Blood_Date'])
# dates['Blood_Date']=dates['Blood_Date']
dates.head()

In [ ]:
dates['Month']=dates['Blood_Date'].apply(lambda x: x.month).astype(int)
dates['Year']=dates['Blood_Date'].apply(lambda x: x.year).astype(int)
dates['Season']=dates['Month'].map({1:'winter',2:'winter',12:'winter',3:'spring',4:'spring',5:'spring',
                                    6:'summer',7:'summer',8:'summer',9:'automn',4:'automn',5:'automn'})
dates['Month_year']=dates[['Month','Year']].astype(str).apply(lambda x: '_'.join(x), axis=1)
dates['Season_year']=dates[['Season','Year']].astype(str).apply(lambda x: '_'.join(x), axis=1)
dates=dates[dates['Year']!=1990]
dates=dates.dropna()
# dates[week]
print dates.shape

dates.to_excel(SEASON_DIR+'AdmissionDates_PNP530.xlsx')
dates.head()

# for col in dates.columns:
#     print dates[col].value_counts(dropna=False)

In [ ]:
for col in dates.columns[1:]:
    print col
    print dates[col].value_counts(dropna=False)
    
    print ''

### generate tables of dates as targets for predictions (dummy variables)

In [ ]:
dates2=dates.iloc[:,1:]
print dates2.shape
print dates2.head()
print dates2.columns
dates_targets=pd.get_dummies(dates2, columns=dates2.columns,prefix=dates2.columns)
dates_targets=dates_targets.loc[:,dates_targets.sum()[dates_targets.sum()>19].index.tolist()]
print dates_targets.shape
dates_targets.index=dates_targets.index.rename('BD')
dates_targets.to_excel(SEASON_DIR+'dates_targets.xlsx')
dates_targets.head()


In [ ]:
#check filtering: expect no results here:
for col in dates_targets.columns:
    if dates_targets[col].sum()<20: print col

In [ ]:
dates.loc['BD406','Blood_Date']-dates.loc['BD228','Blood_Date']

## generate distance table in days

In [ ]:
dates_distMat=pd.DataFrame()
count=0
for i in range(len(dates)):
    if i%10==0:print i
    for j in range(i+1,len(dates)):
        dates_distMat.loc[count,'Sample1']=dates.index[i]
        dates_distMat.loc[count,'Sample2']=dates.index[j]
        dates_distMat.loc[count,'dist']=dates.loc[dates.index[i],'Blood_Date']-dates.loc[dates.index[j],'Blood_Date']
        
        count=count+1
        
dates_distMat['dist2']=np.abs(dates_distMat['dist'] / np.timedelta64(1, 'D'))

            

In [ ]:
dates_distMat.to_pickle(SEASON_DIR+'dates_distMat_PNP530.dat')

### take only balanced old males:

In [ ]:
#get balanced sample lists
with open(SAMPLE_LIST_DIR+'PNP530_balancedAge_males','rb') as fp:
    PNP530_balanced=pickle.load(fp)
with open(SAMPLE_LIST_DIR+'Cardio126_balancedAge_males','rb') as fp:
    Cardio126_balanced=pickle.load(fp)

In [ ]:
dates_distMat=dates_distMat[(dates_distMat['Sample1'].isin(PNP530_balanced))&(dates_distMat['Sample2'].isin(PNP530_balanced))]
print dates_distMat.shape

In [ ]:
dates_distMat.head()

In [ ]:
print len(PNP530_balanced)

In [ ]:
print np.sum(range(57))

In [ ]:
dates_distMat.to_pickle(SEASON_DIR+'dates_distMat_PNP530_balancedOldMales.dat')

In [ ]:
dates_distMat['dist2'].hist()
plt.show()

## cardio126:

### generate table with months and weeks

In [ ]:
dates_cardio=pd.read_excel('%s/TCR_real_data/CardioSamples/phenotypicData/TimeToBlood2_Manual.xlsx' %MyPath).set_index('BD')
dates_cardio=pd.DataFrame(dates_cardio['Admission Date'].dropna())
dates_cardio['Admission Date']==pd.to_datetime(dates_cardio['Admission Date'])

dates_cardio.head(5)

In [ ]:
dates_cardio['Month']=dates_cardio['Admission Date'].apply(lambda x: x.month).astype(int)
dates_cardio['Year']=dates_cardio['Admission Date'].apply(lambda x: x.year).astype(int)
dates_cardio['Season']=dates_cardio['Month'].map({1:'winter',2:'winter',12:'winter',3:'spring',4:'spring',5:'spring',
                                    6:'summer',7:'summer',8:'summer',9:'automn',4:'automn',5:'automn'})
dates_cardio['Month_year']=dates_cardio[['Month','Year']].astype(str).apply(lambda x: '_'.join(x), axis=1)
dates_cardio['Season_year']=dates_cardio[['Season','Year']].astype(str).apply(lambda x: '_'.join(x), axis=1)
dates_cardio=dates_cardio[dates_cardio['Year']!=1990]
dates_cardio=dates_cardio.dropna()
# dates_cardio[week]
print dates_cardio.shape

dates_cardio.to_excel(SEASON_DIR+'Admissiondates_cardio_PNP530.xlsx')
dates_cardio.head()

# for col in dates_cardio.columns:
#     print dates_cardio[col].value_counts(dropna=False)

In [ ]:
dates_cardio['Admission Date'].dtype

In [ ]:
for col in dates_cardio.columns[1:]:
    print col
    print dates_cardio[col].value_counts(dropna=False)
    
    print ''

### generate table with dummy varaibles as target

In [ ]:
dates_cardio2=dates_cardio.iloc[:,1:]
print dates_cardio2.shape
print dates_cardio2.head()
print dates_cardio2.columns
dates_targets_cardio=pd.get_dummies(dates_cardio2, columns=dates_cardio2.columns,prefix=dates_cardio2.columns)
dates_targets_cardio=dates_targets_cardio.loc[:,dates_targets_cardio.sum()[dates_targets_cardio.sum()>6].index.tolist()]
print dates_targets_cardio.shape
dates_targets_cardio.index=dates_targets_cardio.index.rename('BD')
dates_targets_cardio.to_excel(SEASON_DIR+'dates_targets_cardio.xlsx')
dates_targets_cardio.head()


In [ ]:
#check filtering: expect no results here:
for col in dates_targets.columns:
    if dates_targets[col].sum()<20: print col

### generate distance table in days

In [ ]:
dates_cardio.loc['BD950','Admission Date']-dates_cardio.loc['BD951','Admission Date']

In [ ]:
dates_cardio_distMat=pd.DataFrame()
count=0
for i in range(len(dates_cardio)):
    if i%10==0: print i
    for j in range(i+1,len(dates_cardio)):
        dates_cardio_distMat.loc[count,'Sample1']=dates_cardio.index[i]
        dates_cardio_distMat.loc[count,'Sample2']=dates_cardio.index[j]
        dates_cardio_distMat.loc[count,'dist']=dates_cardio.loc[dates_cardio.index[i],'Admission Date']-dates_cardio.loc[dates_cardio.index[j],'Admission Date']
        
        count=count+1
        
dates_cardio_distMat['dist2']=np.abs(dates_cardio_distMat['dist'] / np.timedelta64(1, 'D'))
dates_cardio_distMat 

In [ ]:
dates_cardio_distMat.to_pickle(SEASON_DIR+'dates_distMat_Cardio126.dat')

In [ ]:
dates_cardio_distMat_balanced=dates_cardio_distMat[(dates_cardio_distMat['Sample1'].isin(Cardio126_balanced))&(dates_cardio_distMat['Sample2'].isin(Cardio126_balanced))]
print dates_cardio_distMat_balanced.shape

In [ ]:
dates_cardio_distMat_balanced.to_pickle(SEASON_DIR+'dates_distMat_Cardio126_balancedOldMales.dat')

# generate jaccard distance matrix

In [ ]:
clusterDF=pd.read_pickle('/net/mraid08/export/jafar/Microbiome/Analyses/TCR/TCR_seqs_clusters/newX_onlySeqs_025_085_noNans.dat')
print clusterDF.shape
clusterDF.head()

In [ ]:
clusterDF_binary=(clusterDF>0).astype(int)

## PNP530

In [ ]:
clusterDF_binary_PNP530=clusterDF_binary.loc[PNP530,:]
print clusterDF_binary_PNP530.shape
clusterDF_binary_PNP530.head()

In [ ]:
df=clusterDF_binary_PNP530
metric='jaccard'
generateSquareform=False

clusterDF_jaccard_distMat_PNP530,sqaure=genDistMat(df, metric,generateSquareform)

In [ ]:
clusterDF_jaccard_distMat_PNP530.to_pickle(SEASON_DIR+'clusterDF_jaccard_distMat_PNP530.dat')

In [ ]:
clusterDF_jaccard_distMat_PNP530balancedOldMales=clusterDF_jaccard_distMat_PNP530[(clusterDF_jaccard_distMat_PNP530['sample1'].isin(PNP530_balanced))&(clusterDF_jaccard_distMat_PNP530['sample2'].isin(PNP530_balanced))]
print clusterDF_jaccard_distMat_PNP530balancedOldMales.shape
print clusterDF_jaccard_distMat_PNP530balancedOldMales.head()
clusterDF_jaccard_distMat_PNP530balancedOldMales.to_pickle(SEASON_DIR+'clusterDF_jaccard_distMat_PNP530balancedOldMales.dat')

In [ ]:
clusterDF_jaccard_distMat_PNP530balancedOldMales.sort_values(by='dist').head(20)

In [ ]:
clusterDF_jaccard_distMat_PNP530balancedOldMales.sort_values(by='dist').tail(20)

In [ ]:

clusterDF_jaccard_distMat_PNP530balancedOldMales['dist'].hist(bins=20)
plt.show()

## Cardio126

In [ ]:
clusterDF_binary_Cardio126=clusterDF_binary.loc[Cardio126,:]
print clusterDF_binary_Cardio126.shape
clusterDF_binary_Cardio126.head()

In [ ]:
df=clusterDF_binary_Cardio126
metric='jaccard'
generateSquareform=False

clusterDF_jaccard_distMat_Cardio126,sqaure=genDistMat(df, metric,generateSquareform)

In [ ]:
clusterDF_jaccard_distMat_Cardio126.to_pickle(SEASON_DIR+'clusterDF_jaccard_distMat_NP530.dat')

In [ ]:
clusterDF_jaccard_distMat_Cardio126balancedOldMales=clusterDF_jaccard_distMat_Cardio126[(clusterDF_jaccard_distMat_Cardio126['sample1'].isin(Cardio126_balanced))&(clusterDF_jaccard_distMat_Cardio126['sample2'].isin(Cardio126_balanced))]
clusterDF_jaccard_distMat_Cardio126balancedOldMales.to_pickle(SEASON_DIR+'clusterDF_jaccard_distMat_Cardio126balancedOldMales.dat')

# generate pairwise-sharing matrix

In [ ]:
clusterDF=pd.read_pickle('/net/mraid08/export/jafar/Microbiome/Analyses/TCR/TCR_seqs_clusters/newX_onlySeqs_025_085_noNans.dat')
print clusterDF.shape
clusterDF.head()

In [ ]:
clusterDF_binary=(clusterDF>0).astype(int)

## PNP530

In [ ]:
clusterDF_binary_PNP530=clusterDF_binary.loc[PNP530,:]
print clusterDF_binary_PNP530.shape
clusterDF_binary_PNP530.head()

In [ ]:
clusterDF_binary=clusterDF_binary_PNP530


sharing_simMatrix=pd.DataFrame()
count=0
for i in range(len(clusterDF_binary)):
    if i%10==0:print i
    for j in range(i+1,len(clusterDF_binary)):
        sharing_simMatrix.loc[count,'Sample1']=clusterDF_binary.index[i]
        sharing_simMatrix.loc[count,'Sample2']=clusterDF_binary.index[j]
        sharing_sum=clusterDF_binary.iloc[[i,j],:].sum()
        identical_num=len(sharing_sum[sharing_sum==2])
        sharing_simMatrix.loc[count,'sharing']=identical_num
        
        count=count+1
        
sharing_simMatrix

            

In [ ]:
sharing_simMatrix.to_pickle(SEASON_DIR+'sharing_simMatrix_PNP530.dat')

In [ ]:
sharing_simMatrixbalancedOldMales_PNP530=sharing_simMatrix[(sharing_simMatrix['sample1'].isin(PNP530_balanced))&(sharing_simMatrix['sample2'].isin(PNP530_balanced))]
print sharing_simMatrixbalancedOldMales_PNP530.shape
print sharing_simMatrixbalancedOldMales_PNP530.head()
sharing_simMatrixbalancedOldMales_PNP530.to_pickle(SEASON_DIR+'sharing_simMatrixbalancedOldMales_PNP530.dat')

In [ ]:
sharing_simMatrixbalancedOldMales_PNP530balancedOldMales.sort_values(by='dist').head(20)

In [ ]:
sharing_simMatrixbalancedOldMales_PNP530balancedOldMales.sort_values(by='dist').tail(20)

In [ ]:

sharing_simMatrixbalancedOldMales_PNP530balancedOldMales['sharing'].hist(bins=20)
plt.show()

## Cardio126

In [ ]:
clusterDF_binary_Cardio126=clusterDF_binary.loc[Cardio126,:]
print clusterDF_binary_Cardio126.shape
clusterDF_binary_Cardio126.head()

In [ ]:
df=clusterDF_binary_Cardio126
metric='jaccard'
generateSquareform=False

clusterDF_jaccard_distMat_Cardio126,sqaure=genDistMat(df, metric,generateSquareform)

In [ ]:
clusterDF_jaccard_distMat_Cardio126.to_pickle(SEASON_DIR+'clusterDF_jaccard_distMat_NP530.dat')

In [ ]:
clusterDF_jaccard_distMat_Cardio126balancedOldMales=clusterDF_jaccard_distMat_Cardio126[(clusterDF_jaccard_distMat_Cardio126['sample1'].isin(Cardio126_balanced))&(clusterDF_jaccard_distMat_Cardio126['sample2'].isin(Cardio126_balanced))]
clusterDF_jaccard_distMat_Cardio126balancedOldMales.to_pickle(SEASON_DIR+'clusterDF_jaccard_distMat_Cardio126balancedOldMales.dat')

# calculate correlations between date distances and TCRdf distances:

## PNP530:

### GET DATA:

In [ ]:
dates_distMat_balancedPNP530=pd.read_pickle(SEASON_DIR+'dates_distMat_PNP530_balancedOldMales.dat')
dates_distMat_balancedPNP530=dates_distMat_balancedPNP530.sort_values(by=['Sample1','Sample2'])
dates_distMat_balancedPNP530=dates_distMat_balancedPNP530.rename(columns={'dist2':'days_dist'})
print dates_distMat_balancedPNP530.shape
dates_distMat_balancedPNP530.head()


In [ ]:
dates_distMat_balancedPNP530['days_dist'].hist(bins=20)
plt.show()

In [ ]:
clusterDF_jaccard_distMat_PNP530balancedOldMales=pd.read_pickle(SEASON_DIR+'clusterDF_jaccard_distMat_PNP530balancedOldMales.dat')
# clusterDF_jaccard_distMat_PNP530balancedOldMales=clusterDF_jaccard_distMat_PNP530balancedOldMales.sort_values(by=['sample1','sample2'])
clusterDF_jaccard_distMat_PNP530balancedOldMales=clusterDF_jaccard_distMat_PNP530balancedOldMales.rename(columns={'dist':'TCRjaccard_dist'})
print clusterDF_jaccard_distMat_PNP530balancedOldMales.shape
print clusterDF_jaccard_distMat_PNP530balancedOldMales.head()

In [ ]:
clusterDF_jaccard_distMat_PNP530balancedOldMales.hist()
plt.show()

In [ ]:
merged_PNP530=pd.merge(dates_distMat_balancedPNP530,clusterDF_jaccard_distMat_PNP530balancedOldMales,how='inner',left_on=('Sample1','Sample2'),right_on=('sample1','sample2'))
print merged_PNP530.shape
merged_PNP530.head()

### correlation:

#### all

In [ ]:
r,p=MyPearsonr(merged_PNP530['days_dist'],merged_PNP530['TCRjaccard_dist'])
print r,p

In [ ]:
fig,ax=plt.subplots()
data1=merged_PNP530['days_dist']
data2=merged_PNP530['TCRjaccard_dist']
data1name='Days Distance'
data2name='TCR seq. distance (Jaccard)'



ax, nsamples,r,p,text, handles, labels=plot_corr(data1,data2,data1name,data2name,ax,title=None,corrType='pearson',
              toAnnotate=True,plotTrendLine=True,scatter_kws={'alpha':0.4,'s':100},
               text_kws={'fontsize':'x-large'},linecolor='black')

plt.show()


#### only pairs with less than 2000 days appart

In [ ]:
def plot_corr_days_TCR_dist(mergedDF, max_days_diff,max_jaccard_dist,title):
    
    mergedDF=mergedDF[mergedDF['days_dist']<max_days_diff]
    mergedDF=mergedDF[mergedDF['TCRjaccard_dist']<max_jaccard_dist]
    mergedDF['new_index']=range(len(mergedDF))
    mergedDF=mergedDF.set_index('new_index')
    print mergedDF.shape
    print mergedDF.head()
    
    r,p=MyPearsonr(mergedDF['days_dist'],mergedDF['TCRjaccard_dist'])
    print r,p
    
    fig,ax=plt.subplots()
    data1=mergedDF['days_dist']
    data2=mergedDF['TCRjaccard_dist']
    data1name='Days Distance'
    data2name='TCR seq. distance (Jaccard)'



    ax, nsamples,r,p,text, handles, labels=plot_corr(data1,data2,data1name,data2name,ax,title,corrType='pearson',
                  toAnnotate=True,plotTrendLine=True,scatter_kws={'alpha':0.4,'s':100},
                   text_kws={'fontsize':'x-large'},linecolor='black')

    plt.show()
    
    return

    
    

In [ ]:
mergedDF=merged_PNP530
max_days_diff=100
max_jaccard_dist=0.7
title='PNP530 (max days diff =%s,max_jaccard_dist=%s)' %(max_days_diff,max_jaccard_dist)

plot_corr_days_TCR_dist(mergedDF, max_days_diff,max_jaccard_dist,title)

In [ ]:
mergedDF=merged_PNP530
max_days_diff=1400
max_jaccard_dist=0.62
title='PNP530 (max days diff =%s,max_jaccard_dist=%s)' %(max_days_diff,max_jaccard_dist)

plot_corr_days_TCR_dist(mergedDF, max_days_diff,max_jaccard_dist,title)

## Cardio126:

### GET DATA:

In [ ]:
dates_distMat_balancedCardio126=pd.read_pickle(SEASON_DIR+'dates_distMat_Cardio126_balancedOldMales.dat')
dates_distMat_balancedCardio126=dates_distMat_balancedCardio126.sort_values(by=['Sample1','Sample2'])
dates_distMat_balancedCardio126=dates_distMat_balancedCardio126.rename(columns={'dist2':'days_dist'})
print dates_distMat_balancedCardio126.shape
dates_distMat_balancedCardio126.head()


In [ ]:
dates_distMat_balancedCardio126['days_dist'].hist(bins=20)
plt.show()

In [ ]:
clusterDF_jaccard_distMat_Cardio126balancedOldMales=pd.read_pickle(SEASON_DIR+'clusterDF_jaccard_distMat_Cardio126balancedOldMales.dat')
# clusterDF_jaccard_distMat_Cardio126balancedOldMales=clusterDF_jaccard_distMat_Cardio126balancedOldMales.sort_values(by=['sample1','sample2'])
clusterDF_jaccard_distMat_Cardio126balancedOldMales=clusterDF_jaccard_distMat_Cardio126balancedOldMales.rename(columns={'dist':'TCRjaccard_dist'})
print clusterDF_jaccard_distMat_Cardio126balancedOldMales.shape
print clusterDF_jaccard_distMat_Cardio126balancedOldMales.head()

In [ ]:
clusterDF_jaccard_distMat_Cardio126balancedOldMales.hist()
plt.show()

In [ ]:
merged_Cardio126=pd.merge(dates_distMat_balancedCardio126,clusterDF_jaccard_distMat_Cardio126balancedOldMales,how='inner',left_on=('Sample1','Sample2'),right_on=('sample1','sample2'))
print merged_Cardio126.shape
merged_Cardio126.head()

### correlation:

In [ ]:
mergedDF=merged_Cardio126
max_days_diff=600
max_jaccard_dist=1
title='Cardio126 (max days diff =%s,max_jaccard_dist=%s)' %(max_days_diff,max_jaccard_dist)

plot_corr_days_TCR_dist(mergedDF, max_days_diff,max_jaccard_dist,title)

In [ ]:
mergedDF=merged_Cardio126
max_days_diff=600
max_jaccard_dist=0.75
title='Cardio126 (max days diff =%s,max_jaccard_dist=%s)' %(max_days_diff,max_jaccard_dist)

plot_corr_days_TCR_dist(mergedDF, max_days_diff,max_jaccard_dist,title)

# review seasonality prediction results:

## PNP530-by features:

In [7]:
AUC_threshold=0.6
kappa_threshold=0.2
r_threshold=0
p_r_threshold=0.05

pd.set_option('display.max_colwidth', -1)

dir1='/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/predictions2/seasonality/'
prediction_dirs=['seasonality_XGB50_byTCRfeatures_optByAUC/',
                 'seasonality_XGB_byTCRfeatures_optByAUC']

summaryDF=pd.DataFrame()
problematic_dirs=[]
for pred_dir in prediction_dirs:
#     f2=dir1+pred_dir+'/predictions_df.pkl'
#     predDF=pd.read_pickle(f2)
#     print ('predDF shape=',predDF.shape)
#     print predDF.tail(10)
    
    f1=dir1+pred_dir+'/results_df.pkl'
#     print d
    try:
        df= pd.read_pickle(f1)
        df['dir']=pred_dir.replace('Acute_medication_phenotypes_','')
        summaryDF=pd.concat([summaryDF,df])
    except:
        print ' couldnt get results from %s' %pred_dir
        problematic_dirs.append(pred_dir)
    print pred_dir
    print ('results_df.shape: ',df.shape)
#     print df.head()
        
print ('number of problematic dirs=',len(problematic_dirs))

interesting_cols=['dir','AUC','prevalence','Precision_Recall','kappa']
summaryDF=summaryDF.reset_index().sort_values(by=['AUC'],ascending=False).set_index('index')[interesting_cols]

summaryDF

seasonality_XGB50_byTCRfeatures_optByAUC/
('results_df.shape: ', (34, 14))
seasonality_XGB_byTCRfeatures_optByAUC
('results_df.shape: ', (34, 14))
('number of problematic dirs=', 0)


,dir,AUC,prevalence,Precision_Recall,kappa
index,,,,,
Month_year_3_2015,seasonality_XGB_byTCRfeatures_optByAUC,0.822211,0.063781,0.210682,0.000000
Month_year_3_2015,seasonality_XGB50_byTCRfeatures_optByAUC/,0.800400,0.063781,0.189469,0.110822
Season_year_spring_2015,seasonality_XGB50_byTCRfeatures_optByAUC/,0.768074,0.063781,0.152737,0.047309
Season_year_spring_2015,seasonality_XGB_byTCRfeatures_optByAUC,0.750782,0.063781,0.178473,0.000000
Month_year_2_2014,seasonality_XGB50_byTCRfeatures_optByAUC/,0.722554,0.045558,0.089936,-0.004358
Year_2015,seasonality_XGB_byTCRfeatures_optByAUC,0.719015,0.389522,0.654118,0.306010
Year_2015,seasonality_XGB50_byTCRfeatures_optByAUC/,0.701820,0.389522,0.629934,0.335552
Month_year_4_2015,seasonality_XGB_byTCRfeatures_optByAUC,0.701256,0.056948,0.092880,0.000000
Year_2014,seasonality_XGB_byTCRfeatures_optByAUC,0.643585,0.432802,0.549335,0.159315


## PNP530-by clusters:

In [4]:
AUC_threshold=0.6
kappa_threshold=0.2
r_threshold=0
p_r_threshold=0.05

pd.set_option('display.max_colwidth', -1)

dir1='/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/predictions2/seasonality/'
prediction_dirs=['seasonality_XGB500_byTCRclusters_optByAUC/',
                ]

summaryDF=pd.DataFrame()
problematic_dirs=[]
for pred_dir in prediction_dirs:
#     f2=dir1+pred_dir+'/predictions_df.pkl'
#     predDF=pd.read_pickle(f2)
#     print ('predDF shape=',predDF.shape)
#     print predDF.tail(10)
    
    f1=dir1+pred_dir+'/results_df.pkl'
#     print d
    try:
        df= pd.read_pickle(f1)
        df['dir']=pred_dir.replace('Acute_medication_phenotypes_','')
        summaryDF=pd.concat([summaryDF,df])
    except:
        print ' couldnt get results from %s' %pred_dir
        problematic_dirs.append(pred_dir)
    print pred_dir
    print ('results_df.shape: ',df.shape)
#     print df.head()
        
print ('number of problematic dirs=',len(problematic_dirs))

interesting_cols=['dir','AUC','prevalence','Precision_Recall','kappa']
summaryDF=summaryDF.reset_index().sort_values(by=['AUC'],ascending=False).set_index('index')[interesting_cols]

summaryDF

seasonality_XGB500_byTCRclusters_optByAUC/
('results_df.shape: ', (34, 14))
('number of problematic dirs=', 0)


,dir,AUC,prevalence,Precision_Recall,kappa
index,,,,,
Year_2013,seasonality_XGB500_byTCRclusters_optByAUC/,0.640206,0.113895,0.197829,0.058182
Month_year_6_2014,seasonality_XGB500_byTCRclusters_optByAUC/,0.611561,0.061503,0.208530,0.130555
Season_year_spring_2015,seasonality_XGB500_byTCRclusters_optByAUC/,0.605057,0.063781,0.089872,0.000000
Month_year_3_2015,seasonality_XGB500_byTCRclusters_optByAUC/,0.597410,0.063781,0.087316,0.000000
Month_7,seasonality_XGB500_byTCRclusters_optByAUC/,0.586222,0.152620,0.195107,0.012815
Month_8,seasonality_XGB500_byTCRclusters_optByAUC/,0.584919,0.082005,0.125147,-0.004452
Season_year_summer_2014,seasonality_XGB500_byTCRclusters_optByAUC/,0.584344,0.175399,0.278283,0.048151
Month_year_8_2014,seasonality_XGB500_byTCRclusters_optByAUC/,0.567923,0.056948,0.064203,0.000000
Month_year_2_2014,seasonality_XGB500_byTCRclusters_optByAUC/,0.562888,0.045558,0.063905,-0.004358


## PNP530-by phenotypes:

In [8]:
AUC_threshold=0.6
kappa_threshold=0.2
r_threshold=0
p_r_threshold=0.05

pd.set_option('display.max_colwidth', -1)

dir1='/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/predictions2/seasonality/'
prediction_dirs=['seasonality_XGB_byMajorPhenotypes_optByAUC/',
                ]

summaryDF=pd.DataFrame()
problematic_dirs=[]
for pred_dir in prediction_dirs:
#     f2=dir1+pred_dir+'/predictions_df.pkl'
#     predDF=pd.read_pickle(f2)
#     print ('predDF shape=',predDF.shape)
#     print predDF.tail(10)
    
    f1=dir1+pred_dir+'/results_df.pkl'
#     print d
    try:
        df= pd.read_pickle(f1)
        df['dir']=pred_dir.replace('Acute_medication_phenotypes_','')
        summaryDF=pd.concat([summaryDF,df])
    except:
        print ' couldnt get results from %s' %pred_dir
        problematic_dirs.append(pred_dir)
    print pred_dir
    print ('results_df.shape: ',df.shape)
#     print df.head()
        
print ('number of problematic dirs=',len(problematic_dirs))

interesting_cols=['dir','AUC','prevalence','Precision_Recall','kappa']
summaryDF=summaryDF.reset_index().sort_values(by=['AUC'],ascending=False).set_index('index')[interesting_cols]

summaryDF

 couldnt get results from seasonality_XGB_byMajorPhenotypes_optByAUC/
seasonality_XGB_byMajorPhenotypes_optByAUC/
('results_df.shape: ', (34, 14))
('number of problematic dirs=', 1)


KeyError: 'AUC'

## Cardio126-by features:

In [5]:
AUC_threshold=0.6
kappa_threshold=0.2
r_threshold=0
p_r_threshold=0.05

pd.set_option('display.max_colwidth', -1)

dir1='/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/predictions2/seasonality/'
prediction_dirs=['seasonality_XGB50_byTCRfeatures_optByAUC_Cardio126/',
                 'seasonality_XGB_byTCRfeatures_optByAUC_Cardio126']

summaryDF=pd.DataFrame()
problematic_dirs=[]
for pred_dir in prediction_dirs:
#     f2=dir1+pred_dir+'/predictions_df.pkl'
#     predDF=pd.read_pickle(f2)
#     print ('predDF shape=',predDF.shape)
#     print predDF.tail(10)
    
    f1=dir1+pred_dir+'/results_df.pkl'
#     print d
    try:
        df= pd.read_pickle(f1)
        df['dir']=pred_dir.replace('Acute_medication_phenotypes_','')
        summaryDF=pd.concat([summaryDF,df])
    except:
        print ' couldnt get results from %s' %pred_dir
        problematic_dirs.append(pred_dir)
    print pred_dir
    print ('results_df.shape: ',df.shape)
#     print df.head()
        
print ('number of problematic dirs=',len(problematic_dirs))

interesting_cols=['dir','AUC','prevalence','Precision_Recall','kappa']
summaryDF=summaryDF.reset_index().sort_values(by=['AUC'],ascending=False).set_index('index')[interesting_cols]

summaryDF

seasonality_XGB50_byTCRfeatures_optByAUC_Cardio126/
('results_df.shape: ', (25, 14))
seasonality_XGB_byTCRfeatures_optByAUC_Cardio126
('results_df.shape: ', (25, 14))
('number of problematic dirs=', 0)


,dir,AUC,prevalence,Precision_Recall,kappa
index,,,,,
Year_2018,seasonality_XGB_byTCRfeatures_optByAUC_Cardio126,0.733848,0.082474,0.165842,0.000000
Year_2018,seasonality_XGB50_byTCRfeatures_optByAUC_Cardio126/,0.699438,0.082474,0.169439,0.000000
Month_4,seasonality_XGB_byTCRfeatures_optByAUC_Cardio126,0.697674,0.113402,0.265998,0.000000
Month_1,seasonality_XGB_byTCRfeatures_optByAUC_Cardio126,0.696032,0.072165,0.245548,0.000000
Year_2017,seasonality_XGB50_byTCRfeatures_optByAUC_Cardio126/,0.693820,0.917526,0.956151,0.000000
Month_year_4_2017,seasonality_XGB_byTCRfeatures_optByAUC_Cardio126,0.690805,0.103093,0.421803,0.000000
Year_2017,seasonality_XGB_byTCRfeatures_optByAUC_Cardio126,0.663624,0.917526,0.960782,0.000000
Season_year_automn_2017,seasonality_XGB_byTCRfeatures_optByAUC_Cardio126,0.649425,0.402062,0.504354,0.061807
Month_5,seasonality_XGB50_byTCRfeatures_optByAUC_Cardio126/,0.636275,0.123711,0.173012,0.000000


## Cardio126-by clusters:

In [6]:
AUC_threshold=0.6
kappa_threshold=0.2
r_threshold=0
p_r_threshold=0.05

pd.set_option('display.max_colwidth', -1)

dir1='/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/predictions2/seasonality/'
prediction_dirs=['seasonality_XGB500_byTCRclusters_optByAUC_Cardio126/',
                ]

summaryDF=pd.DataFrame()
problematic_dirs=[]
for pred_dir in prediction_dirs:
#     f2=dir1+pred_dir+'/predictions_df.pkl'
#     predDF=pd.read_pickle(f2)
#     print ('predDF shape=',predDF.shape)
#     print predDF.tail(10)
    
    f1=dir1+pred_dir+'/results_df.pkl'
#     print d
    try:
        df= pd.read_pickle(f1)
        df['dir']=pred_dir.replace('Acute_medication_phenotypes_','')
        summaryDF=pd.concat([summaryDF,df])
    except:
        print ' couldnt get results from %s' %pred_dir
        problematic_dirs.append(pred_dir)
    print pred_dir
    print ('results_df.shape: ',df.shape)
#     print df.head()
        
print ('number of problematic dirs=',len(problematic_dirs))

interesting_cols=['dir','AUC','prevalence','Precision_Recall','kappa']
summaryDF=summaryDF.reset_index().sort_values(by=['AUC'],ascending=False).set_index('index')[interesting_cols]

summaryDF

seasonality_XGB500_byTCRclusters_optByAUC_Cardio126/
('results_df.shape: ', (25, 14))
('number of problematic dirs=', 0)


,dir,AUC,prevalence,Precision_Recall,kappa
index,,,,,
Month_1,seasonality_XGB500_byTCRclusters_optByAUC_Cardio126/,0.673016,0.072165,0.096667,0.000000
Month_year_12_2017,seasonality_XGB500_byTCRclusters_optByAUC_Cardio126/,0.664326,0.082474,0.130112,0.000000
Season_summer,seasonality_XGB500_byTCRclusters_optByAUC_Cardio126/,0.617140,0.298969,0.383512,0.080278
Season_year_summer_2017,seasonality_XGB500_byTCRclusters_optByAUC_Cardio126/,0.614907,0.288660,0.416310,0.148548
Month_12,seasonality_XGB500_byTCRclusters_optByAUC_Cardio126/,0.606742,0.082474,0.127313,0.000000
Year_2017,seasonality_XGB500_byTCRclusters_optByAUC_Cardio126/,0.602528,0.917526,0.951342,0.000000
Month_6,seasonality_XGB500_byTCRclusters_optByAUC_Cardio126/,0.590659,0.134021,0.153947,0.000000
Year_2018,seasonality_XGB500_byTCRclusters_optByAUC_Cardio126/,0.577247,0.082474,0.113930,0.000000
Month_year_6_2017,seasonality_XGB500_byTCRclusters_optByAUC_Cardio126/,0.568681,0.134021,0.141714,0.000000


## Cardio126-by phenotypes:

In [9]:
AUC_threshold=0.6
kappa_threshold=0.2
r_threshold=0
p_r_threshold=0.05

pd.set_option('display.max_colwidth', -1)

dir1='/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/predictions2/seasonality/'
prediction_dirs=['seasonality_XGB_byMajorPhenotypes_optByAUC_Cardio126/',
                ]

summaryDF=pd.DataFrame()
problematic_dirs=[]
for pred_dir in prediction_dirs:
#     f2=dir1+pred_dir+'/predictions_df.pkl'
#     predDF=pd.read_pickle(f2)
#     print ('predDF shape=',predDF.shape)
#     print predDF.tail(10)
    
    f1=dir1+pred_dir+'/results_df.pkl'
#     print d
    try:
        df= pd.read_pickle(f1)
        df['dir']=pred_dir.replace('Acute_medication_phenotypes_','')
        summaryDF=pd.concat([summaryDF,df])
    except:
        print ' couldnt get results from %s' %pred_dir
        problematic_dirs.append(pred_dir)
    print pred_dir
    print ('results_df.shape: ',df.shape)
#     print df.head()
        
print ('number of problematic dirs=',len(problematic_dirs))

interesting_cols=['dir','AUC','prevalence','Precision_Recall','kappa']
summaryDF=summaryDF.reset_index().sort_values(by=['AUC'],ascending=False).set_index('index')[interesting_cols]

summaryDF

seasonality_XGB_byMajorPhenotypes_optByAUC_Cardio126/
('results_df.shape: ', (25, 14))
('number of problematic dirs=', 0)


,dir,AUC,prevalence,Precision_Recall,kappa
index,,,,,
Month_12,seasonality_XGB_byMajorPhenotypes_optByAUC_Cardio126/,0.702247,0.082474,0.151540,0.000000
Month_year_12_2017,seasonality_XGB_byMajorPhenotypes_optByAUC_Cardio126/,0.683989,0.082474,0.176668,0.000000
Month_year_7_2017,seasonality_XGB_byMajorPhenotypes_optByAUC_Cardio126/,0.632353,0.123711,0.153933,-0.036641
Month_1,seasonality_XGB_byMajorPhenotypes_optByAUC_Cardio126/,0.605556,0.072165,0.086155,-0.018373
Month_7,seasonality_XGB_byMajorPhenotypes_optByAUC_Cardio126/,0.559982,0.134021,0.141065,-0.019520
Month_9,seasonality_XGB_byMajorPhenotypes_optByAUC_Cardio126/,0.553446,0.185567,0.213155,-0.038544
Year_2017,seasonality_XGB_byMajorPhenotypes_optByAUC_Cardio126/,0.511236,0.917526,0.923907,0.000000
Month_year_5_2017,seasonality_XGB_byMajorPhenotypes_optByAUC_Cardio126/,0.506342,0.113402,0.161124,0.000000
Season_winter,seasonality_XGB_byMajorPhenotypes_optByAUC_Cardio126/,0.492240,0.195876,0.184229,0.000000
